In [17]:
# import pandas
import pandas as pd

In [18]:
# อ่านไฟล์ Dataset

# ระบุ path
path = "Data.txt"

# อ่านมาเป็น df
df = pd.read_csv(path, header=None)

# ตรวจดูตัวอย่าง 5 แถวแรก
print(df.head())

# ดูขนาดตาราง (แถว, คอลัมน์)
print(df.shape)


       0   1       2                 3                   4      5
0  White  39    Male         State-gov       Never-married  <=50K
1  White  50    Male  Self-emp-not-inc  Married-civ-spouse  <=50K
2  White  38    Male           Private            Divorced  <=50K
3  Black  53    Male           Private  Married-civ-spouse  <=50K
4  Black  28  Female           Private  Married-civ-spouse  <=50K
(30725, 6)


In [19]:
# กำหนด header
cols = ["Race", "Age", "Sex", "Workclass", "Marital-status", "Income/year"]

# กำหนด header ให้ DataFrame
df.columns = cols

# ตรวจสอบ
print(df.head())

    Race  Age     Sex         Workclass      Marital-status Income/year
0  White   39    Male         State-gov       Never-married       <=50K
1  White   50    Male  Self-emp-not-inc  Married-civ-spouse       <=50K
2  White   38    Male           Private            Divorced       <=50K
3  Black   53    Male           Private  Married-civ-spouse       <=50K
4  Black   28  Female           Private  Married-civ-spouse       <=50K


In [20]:
# ตัด attribute "Income/year" ออก 
df.drop(columns=["Income/year"], inplace=True)

# ตรวจสอบ
print(df.head())

    Race  Age     Sex         Workclass      Marital-status
0  White   39    Male         State-gov       Never-married
1  White   50    Male  Self-emp-not-inc  Married-civ-spouse
2  White   38    Male           Private            Divorced
3  Black   53    Male           Private  Married-civ-spouse
4  Black   28  Female           Private  Married-civ-spouse


In [21]:
# กำหนดประเภท Attribute
SENSITIVE_ATTRS = ["Workclass"] 
QI_ATTRS = [c for c in df.columns if c not in SENSITIVE_ATTRS]

In [22]:
# กำหนด VGH

# AGE
def vgh_age(a, level: int):
    # level: 0=exact, 1=1-25/26-50/51-75/76-100, 2=1-50/51-100, 3='*'
    if level == 3: return "*"
    a = int(a)
    if level == 0: return str(a)
    if level == 1:
        if   1 <= a <= 25:  return "1-25"
        if  26 <= a <= 50:  return "26-50"
        if  51 <= a <= 75:  return "51-75"
        return "76-100"
    if level == 2: return "1-50" if a <= 50 else "51-100"

# SEX
def vgh_sex(s, level: int):
    # level: 0=Female/Male, 1='*'
    return "*" if level == 1 else str(s)

# RACE
def vgh_race(r, level: int):
    # level: 0=5 หมวด, 1=รวม {Amer-Indian-Eskimo,Other}, 2=White/Non-White, 3='*'
    r = str(r)
    if level == 3: return "*"
    if level == 2: return "White" if r == "White" else "Non-White"
    if level == 1: return "Amer-Indian-Eskimo+Other" if r in {"Amer-Indian-Eskimo","Other"} else r
    return r  # level 0

# MARITAL
MARRIED = {"Married-civ-spouse","Married-AF-spouse"}
PREV_MARRIED = {"Divorced","Separated","Widowed","Married-spouse-absent"}

def vgh_marital(m, level: int):
    # level: 0=7 ค่าเดิม, 1=Married/Previously-married/Never-married, 2=Married/Not-married, 3='*'
    m = str(m)
    if level == 3: return "*"
    if level == 2: return "Married" if m in MARRIED else "Not-married"
    if level == 1:
        if m in MARRIED: return "Married"
        if m in PREV_MARRIED: return "Previously-married"
        return "Never-married"
    return m  # level 0

In [23]:
# กำหนด K
TARGET_K = 4

In [ ]:
# เราจะทำ "local recoding" เลยต้องเก็บค่าต้นฉบับไว้ + มีคอลัมน์เก็บระดับของแต่ละ QI
for col in QI_ATTRS:
    df[col + "_orig"] = df[col]   # สำรองค่าต้นฉบับ
# ระดับเริ่มต้น = 0 (ใช้ L0 ของแต่ละ VGH/DGH)
df["Age_lvl"] = 0
df["Race_lvl"] = 0
df["Marital-status_lvl"] = 0
df["Sex_lvl"] = 0

# จำกัดระดับสูงสุดของแต่ละคอลัมน์ (ตาม VGH/DGH ที่กำหนด)
MAX_LVL = {
    "Age": 3,              # 0..3
    "Race": 3,             # 0..3
    "Marital-status": 3,   # 0..3
    "Sex": 1,              # 0..1
}

# ตัวช่วยเรียก generalizer ต่อค่าเดียว
def generalize_one(attr, value, level):
    if attr == "Age":
        return vgh_age(value, level)
    if attr == "Race":
        return vgh_race(value, level)
    if attr == "Marital-status":
        return vgh_marital(value, level)
    if attr == "Sex":
        return vgh_sex(value, level)
    return value  # เผื่อมีคอลัมน์อื่น

# คำนวณค่า generalized ของ QIs ให้แถวๆ หนึ่ง จาก "ค่าต้นฉบับ" + "ระดับของแถวนั้น"
def apply_generalization_for_rows(row_subset):
    """
    row_subset: DataFrame (subset ของ df) ที่ต้องการคำนวณค่า generalized ใหม่
    คืนค่า DataFrame ที่มีคอลัมน์ QIs เป็นค่าที่ generalize แล้ว
    """
    def _row_to_series(r):
        return pd.Series({
            "Race": generalize_one("Race", r["Race_orig"], r["Race_lvl"]),
            "Age": generalize_one("Age", r["Age_orig"], r["Age_lvl"]),
            "Sex": generalize_one("Sex", r["Sex_orig"], r["Sex_lvl"]),
            "Marital-status": generalize_one("Marital-status", r["Marital-status_orig"], r["Marital-status_lvl"]),
        })
    return row_subset.apply(_row_to_series, axis=1)

# เริ่มรอบแรก: sync ค่าทั้งตารางให้เป็นระดับ 0 (คือค่าดิบ)
df.loc[:, QI_ATTRS] = apply_generalization_for_rows(df)

# ---- วนทำตามขั้นตอนที่กำหนด ----
# ลำดับการ "ยกระดับทีละ 1" (แนะนำ: Age -> Race -> Marital-status -> Sex)
BUMP_ORDER = ["Age", "Race", "Marital-status", "Sex"]

max_iters = 50  # กันวนไม่รู้จบ
for it in range(max_iters):
    # 1) จัดกลุ่ม EC และนับขนาด
    ec_sizes = df.groupby(QI_ATTRS, dropna=False).size().rename("ec_size")
    # ผูกขนาด EC กลับเข้าแต่ละแถว
    df = df.merge(ec_sizes.reset_index(), on=QI_ATTRS, how="left")

    # 2) ตรวจสอบ k ขั้นต่ำ
    current_min_k = int(df["ec_size"].min())
    print(f"[Iter {it}] min-k = {current_min_k}, rows = {len(df)}")

    # ถ้าทุก EC ผ่านแล้ว (>= TARGET_K) ก็จบ
    if current_min_k >= TARGET_K:
        print("All ECs meet k-anonymity.")
        df.drop(columns=["ec_size"], inplace=True)  # ทำความสะอาด
        break

    # 3) แยก EC ที่ "ยังไม่ผ่าน"
    fail_mask = df["ec_size"] < TARGET_K
    num_fail = int(fail_mask.sum())
    print(f"  - rows not meeting k: {num_fail}")

    # ถ้าแถวที่ยังไม่ผ่านมีน้อยกว่า k -> ลบทิ้งตามกติกา แล้วจบ
    if num_fail < TARGET_K:
        print(f"  - rows not meeting k < {TARGET_K}  -> drop them")
        df = df.loc[~fail_mask].copy()
        df.drop(columns=["ec_size"], inplace=True)
        break

    # 4) ยกระดับ "เฉพาะแถวที่ยังไม่ผ่าน" ขึ้นทีละ 1 ตามลำดับ BUMP_ORDER
    #    - ต่อแถว เราจะยกระดับแค่ 1 คอลัมน์/รอบ (คอลัมน์แรกที่ยังไม่สุด MAX_LVL)
    to_bump = fail_mask.copy()
    bumped_any = False
    for attr in BUMP_ORDER:
        lvl_col = attr + "_lvl"
        can_bump_here = to_bump & (df[lvl_col] < MAX_LVL[attr])
        if can_bump_here.any():
            # เพิ่มระดับขึ้น 1 เฉพาะแถวที่ยังบั้มได้
            df.loc[can_bump_here, lvl_col] = df.loc[can_bump_here, lvl_col] + 1
            # คำนวณค่าที่ generalize ใหม่ให้กับแถวเหล่านี้
            df.loc[can_bump_here, QI_ATTRS] = apply_generalization_for_rows(df.loc[can_bump_here, :])
            # แถวไหนถูกบั้มแล้ว ไม่ต้องพิจารณาอีกในรอบนี้
            to_bump = to_bump & (~can_bump_here)
            bumped_any = True
        # ถ้าทุกแถวที่ยังไม่ผ่านถูกบั้มแล้ว ก็หยุดหา attr ต่อไป
        if not to_bump.any():
            break

    # 5) ทำความสะอาดคอลัมน์ ec_size ก่อนวนรอบใหม่
    df.drop(columns=["ec_size"], inplace=True)

    # กรณีรอบนี้ไม่มีแถวไหนบั้มได้แล้ว (ทุกคนทะลุ MAX_LVL แต่ยังไม่ถึง k): ลบทิ้งส่วนที่ยังไม่ผ่าน
    if not bumped_any:
        print("  - No more generalization possible; suppress remaining failing rows.")
        df = df.loc[~fail_mask].copy()
        break

else:
    # ถ้าออกจากลูปเพราะครบ max_iters
    print("Reached max iterations; stopping.")

# ---- สรุปผล ----
# print("\n=== Summary ===")
# final_ec_sizes = df.groupby(QI_ATTRS, dropna=False).size().rename("count").reset_index()
# print(f"Rows left: {len(df)}")
# print(f"EC count: {len(final_ec_sizes)}")
# print(final_ec_sizes.head(10))
print(df.tail(20))


C:\Users\vrzo2\AppData\Local\Temp\ipykernel_14080\1074809989.py:46: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['39' '50' '38' ... '58' '22' '52']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[:, QI_ATTRS] = apply_generalization_for_rows(df)


[Iter 0] min-k = 1, rows = 30725
  - rows not meeting k: 1437
[Iter 1] min-k = 1, rows = 30725
  - rows not meeting k: 118
[Iter 2] min-k = 1, rows = 30725
  - rows not meeting k: 104
[Iter 3] min-k = 1, rows = 30725
  - rows not meeting k: 76
[Iter 4] min-k = 1, rows = 30725
  - rows not meeting k: 54
[Iter 5] min-k = 1, rows = 30725
  - rows not meeting k: 27
[Iter 6] min-k = 1, rows = 30725
  - rows not meeting k: 14
[Iter 7] min-k = 3, rows = 30725
  - rows not meeting k: 6
[Iter 8] min-k = 3, rows = 30725
  - rows not meeting k: 6
[Iter 9] min-k = 3, rows = 30725
  - rows not meeting k: 6
[Iter 10] min-k = 4, rows = 30725
All ECs meet k-anonymity.
                  Race    Age     Sex         Workclass  \
0                White     39    Male         State-gov   
1                White     50    Male  Self-emp-not-inc   
2                White     38    Male           Private   
3                Black     53    Male           Private   
4                Black     28  Female       